# Development practises


## Topics
- naming conventions
- pretty functions
- multiple dispatch
- variable scope
    - arrays as an exception
- metaprogramming (see Bonus notebook)

## Naming conventions in Julia

- Word separation can be indicated by underscores (`_`), but use of underscores is discouraged unless the name would be hard to read otherwise.
- Names of Types begin with a capital letter and word separation is shown with CamelCase instead of underscores.
- Names of functions and macros are in lower case, without underscores.  
- Functions that modify their inputs have names that end in `!`. These functions are sometimes called mutating functions or in-place functions.


## Making functions pretty: optional arguments
You can define functions with optional arguments, so that the function can use sensible defaults if specific values aren't supplied. You provide a default symbol and value in the argument list

In [ ]:
function xyzpos(x, y, z=0)
    println("$x, $y, $z")
end

In [ ]:
xyzpos(0,0)
xyzpos(0,0,1)

## Making functions pretty: keyword arguments
When you write a function with a long list of arguments like this:
```julia
function f(p, q, r, s, t, u)
...
end
```
sooner or later, you will forget the order in which you have to supply the arguments. 

You can avoid this problem by using keywords to label arguments. Use a semicolon (`;`) after the function's unlabelled arguments, and follow it with one or more keyword=value pairs:

In [ ]:
function f(p, q ; radius = 4, message = "hello")
  println("p is $p")
  println("q is $q")
  return "radius => $radius, message => $message"
end
f(1,2)
f("a", "b", r=pi, s=22//7)

## Advanced: Functions with variable number of arguments
Functions can be defined so that they can accept any number of arguments:

In [ ]:
function fvar(args...)
    println("you supplied $(length(args)) arguments")
    for arg in args
       println(" argument ", arg)
    end
end
fvar()
fvar(64)
fvar(64, 64, 55)

The three dots indicate the **splat**. Here it means "any", including "none". 

## Advanced: Parametric methods
Method definitions can optionally have type parameters qualifying the signature:

In [ ]:
function same_type(x::T, y::T) where {T}
    true
end

function same_type(x,y)
    false
end

The first method applies whenever both arguments are of the same concrete type, regardless of what type that is, while the second method acts as a catch-all, covering all other cases. Thus, overall, this defines a boolean function that checks whether its two arguments are of the same type

In [ ]:
same_type(1,2)

In [ ]:
same_type(1, 2.0)

In [ ]:
same_type(1.0, 2.0)

In [ ]:
same_type("foo", 2.0)

In [ ]:
same_type(Int32(1), Int64(2))

## Scope of variables
- Global scope
    - Module specific (namespaces)
- Local scopes
    - functions, for's, while's,...

## Local scope
A new local scope is introduced by most code-blocks.
    
A local scope usually inherits all the variables from its parent scope, both for reading and writing. 

A newly introduced variable in a local scope does not back-propagate to its parent scope. For example, here the z is not introduced into the top-level scope:

In [ ]:
for i = 1:10
    z = 1
end
z

Function definitions are also in their own local scope. 

They do, however, inherit from their parent scope.

In [ ]:
x, y = 1, 2
function foo()
    x = 2 #assignment introduces a new local
    return x + y # y refers to the global scope!
end

In [ ]:
foo()

In [ ]:
x

An explicit `global` is needed to assign to a global variable:

In [ ]:
x = 1
function foobar()
    global x = 2
    return x + y # Now x is also in global scope!
end

In [ ]:
foobar()

In [ ]:
x

## Exception: Elements of a global array are global
There is an important exception to these rules: arrays.

Changing an elements of an array in global scope is done in the global scope. 

In [ ]:
arr = [1,2,3]
function oops()
    arr[2] = 10
    arrr = [1,2,3]
    
    return "woops"
end

In [ ]:
oops()

In [ ]:
arr

In [ ]:
arrr

## Constants
A common use of variables is giving names to specific, unchanging values. 

Such variables are only assigned once. This intent can be conveyed to the compiler using the `const` keyword:

In [ ]:
const e  = 2.71828182845904523536

It is difficult for the compiler to optimize code involving global variables, since their values (or even their types) might change at almost any time. If a global variable will not change, adding a const declaration solves this performance problem.

## Summary
- Writing pretty code is a good thing
    - see also the official [style guide](https://docs.julialang.org/en/v1/manual/style-guide/)
- remember the index ordering in loops!
    - first index changes fastest
- take advantage of multiple dispatch
    - this is what makes Julia fast